In [1]:
import numpy as np
import pyvista as pv
import torch
import matplotlib.pyplot as plt

In [ ]:
maskUp = 1.69
maskBottom = 2.92
maskDeep = 5.95

diameter = 60
maskUp_sim = maskUp/maskBottom*diameter
maskDeep_sim = maskDeep/maskBottom*diameter
print(maskUp_sim)
print(maskDeep_sim)

In [ ]:
maskUp = 2.09
maskBottom = 2.6
maskDeep = 2.808

diameter = 60
maskUp_sim = maskUp/maskBottom*diameter
maskDeep_sim = maskDeep/maskBottom*diameter
print(maskUp_sim)
print(maskDeep_sim)

In [ ]:
bottom = 100
height = bottom + int(maskDeep_sim)

print('height',height)
for i in range(int(diameter/2 - int(maskUp_sim/2))):
    print(diameter/2 - i)
    print('deep', int(bottom + maskDeep_sim/(diameter/2 - maskUp_sim/2)*(i)))
    print(int(maskDeep_sim/(diameter/2 - maskUp_sim/2)))

In [ ]:
film = np.zeros((100, 100, 200, 3))

bottom = 100
height = bottom + int(maskDeep_sim)

density = 10

sphere = np.ones((100, 100, 200), dtype=bool)

diameter = 60

center = 50
for k in range(int(diameter/2 - int(maskUp_sim/2))):
    # print(diameter/2 - k)
    radius = diameter/2 - k
    # print('deep', int(bottom + maskDeep_sim/(diameter/2 - maskUp_sim/2)*k))
    bottom_step = int(bottom + maskDeep_sim/(diameter/2 - maskUp_sim/2)*k)
    for i in range(sphere.shape[0]):
        for j in range(sphere.shape[1]):
            if np.abs(i-center)*np.abs(i-center) + np.abs(j-center)*np.abs(j-center) < radius*radius:
                sphere[i, j, bottom_step:bottom_step+int(maskDeep_sim/(diameter/2 - maskUp_sim/2))] = 0

film[sphere, 2] = density
film[:, :, height:, :] = 0
film[:, :, 0:bottom, 0] = density # bottom
film[:, :, 0:bottom, 1] = 0 # bottom
film[:, :, 0:bottom, 2] = 0 # bottom

In [2]:
# vertical mask
film = np.zeros((100, 100, 160, 3))

bottom = 100
height = 140

density = 10

sphere = np.ones((100, 100, 160), dtype=bool)

radius = 30

center = 50
for i in range(sphere.shape[0]):
    for j in range(sphere.shape[1]):
        if np.abs(i-center)*np.abs(i-center) + np.abs(j-center)*np.abs(j-center) < radius*radius:
            sphere[i, j, bottom:height] = 0

film[sphere, 2] = density
film[:, :, height:, :] = 0
film[:, :, 0:bottom, 0] = density # bottom
film[:, :, 0:bottom, 1] = 0 # bottom
film[:, :, 0:bottom, 2] = 0 # bottom

In [ ]:
depo1 = torch.Tensor(film[:, :50, :, 1]>0).to_sparse()
depo1 = depo1.indices().numpy().T

substrute = torch.Tensor(film[:, :50, :,0]!=0).to_sparse()
substrute = substrute.indices().numpy().T

mask = torch.Tensor(film[:, :50, :,2]!=0).to_sparse()
mask = mask.indices().numpy().T

maskmesh = pv.PolyData(mask)
maskmesh["radius"] = np.ones(mask.shape[0])*0.5

depomesh = pv.PolyData(depo1)
depomesh["radius"] = np.ones(depo1.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(substrute)
submesh["radius"] = np.ones(substrute.shape[0])*0.5

# Progress bar is a new feature on master branch
maskglyphed = maskmesh.glyph(scale="radius", geom=geom) # progress_bar=True)
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
p.add_mesh(maskglyphed, color='green')
# p.add_mesh(depoglyphed, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

In [3]:
etchfilm = film

In [4]:
import Multi_Species_bosch_etching_countByReaction_mirror

In [5]:
logname = 'Multi_species_benchmark_1013_hole'

testEtch = Multi_Species_bosch_etching_countByReaction_mirror.etching(inputMethod='bunch', depo_or_etching='etching', 
                etchingPoint = np.array([center, center, bottom-30]),depoPoint = np.array([center, center, bottom-30]),
                density=density, center_with_direction=np.array([[50,50,75]]), 
                range3D=np.array([[0, 100, 0, 100, 0, 150]]), InOrOut=[1], yield_hist=np.array([None]),
                maskTop=40, maskBottom=98, maskStep=10, maskCenter=[50, 50],mirrorGap=5,
                reaction_type=False, param = [1.6, -0.7],n=1,
                celllength=1e-5, kdtreeN=5, filmKDTree=np.array([[2, 0], [3, 1]]),weight=-0.2, tstep=1e-5,
                substrateTop=bottom,posGeneratorType='top', logname=logname)

In [7]:
etching = np.load('./bosch_data_1009_ratio08/bosch_sf_step_3_c4f8.npy')

In [8]:
indice_c3 = np.logical_and(etching[:, :, :, 1]>9, etching[:, :, :, 1]<10)
etching[indice_c3, 1] = 10

indice_step2 = etching[:, :, :, 0] != 10
etching[indice_step2, 0] = 0
indice_c2 = etching[:, :, :, 1] != 0
etching[indice_c2, 1] = 10

In [6]:
cicle = 100
celllength=1e-5
parcel = np.array([[95*celllength, 95*celllength, 159*celllength, 0, 0, 1, 95, 95, 159, 0]])
step1 = testEtch.inputParticle(etchfilm, parcel, 'maxwell', 0, int(2e4),int(4e5),2, 10, 1)
np.save('./bosch_data_1015_test/bosch_sf_step_sf01', etchfilm)

particle input:  99%|█████████████████████████████████████████████▌| 99.0/100 [10:17<00:06, 6.23s/B]


In [ ]:
cicle = 100
celllength=1e-5
parcel = np.array([[95*celllength, 95*celllength, 159*celllength, 0, 0, 1, 95, 95, 159, 0]])

for i in range(cicle):
    step1 = testEtch.inputParticle(etchfilm, parcel, 'maxwell', 0, int(2e4),int(4e5),2, 10, i)
    indice_step1 = etchfilm[:, :, :, 0] >= 9
    etchfilm[indice_step1, 0] = 10
    depth = step1[1] 
    parcel = np.array([[95*celllength, 95*celllength, 159*celllength, 0, 0, 1, 95, 95, 159, 0]])
    if depth < 50:
        film = np.zeros((100, 100, etchfilm.shape[2] + 50, 3))
        film[:, :, 50:] = etchfilm
        film[:, :, :50, 0] = 10
        etchfilm = film
        parcel[:, 2] += 50*celllength
        parcel[:, 8] += 50
    print('film:', etchfilm.shape)
    np.save('./bosch_data_1013_ratio01_hole/bosch_sf_step_{}_sf'.format(i), etchfilm)
    step2 = testEtch.inputParticle(etchfilm, parcel, 'maxwell', 1, int(2e4),int(8e4), 1, 10, i)
    parcel = np.array([[95*celllength, 95*celllength, 159*celllength, 0, 0, 1, 95, 95, 159, 0]])
    indice_step2 = etchfilm[:, :, :, 0] < 0
    etchfilm[indice_step2, 0] = 0
    indice_c2 = etchfilm[:, :, :, 1] != 0
    etchfilm[indice_c2, 1] = 10
    np.save('./bosch_data_1013_ratio01_hole/bosch_sf_step_{}_c4f8'.format(i), etchfilm)
    step3 = testEtch.inputParticle(etchfilm, parcel, 'updown', 2, int(2e4),int(6e4), 4, 10, i)
    parcel = np.array([[95*celllength, 95*celllength, 159*celllength, 0, 0, 1, 95, 95, 159, 0]])
    indice_c3 = np.logical_and(etchfilm[:, :, :, 1]>9, etchfilm[:, :, :, 1]<10)
    etchfilm[indice_c3, 1] = 10
    indice_step2 = etchfilm[:, :, :, 0] != 10
    etchfilm[indice_step2, 0] = 0
    indice_c2 = etchfilm[:, :, :, 1] != 0
    etchfilm[indice_c2, 1] = 10
    np.save('./bosch_data_1013_ratio01_hole/bosch_sf_step_{}_Ar'.format(i), etchfilm)

In [32]:
np.save('./bosch_data_1001_ratio08/bosch_sf_stepTest_c4f8', etchfilm)

In [17]:
indice_c3 = np.logical_and(etching[:, :, :, 1]>9, etching[:, :, :, 1]<10)
etching[indice_c3, 1] = 10

indice_step2 = etching[:, :, :, 0] != 10
etching[indice_step2, 0] = 0
indice_c2 = etching[:, :, :, 1] != 0
etching[indice_c2, 1] = 10

In [9]:
etching = np.load('./bosch_data_1015_test/bosch_sf_step_sf.npy')

In [10]:
# cyan = torch.Tensor(np.logical_and(etching[:, :50, :,1]==10, etching[:, :50, :,0]==0)).to_sparse()
cyan = torch.Tensor(etching[:, :50, :,1]!=0).to_sparse()
cyan = cyan.indices().numpy().T

gray = torch.Tensor(etching[:, :50, :,0]!=0).to_sparse()
gray = gray.indices().numpy().T

mask = torch.Tensor(etching[:, :50, :,2]!=0).to_sparse()
mask = mask.indices().numpy().T

maskmesh = pv.PolyData(mask)
maskmesh["radius"] = np.ones(mask.shape[0])*0.5

depomesh = pv.PolyData(cyan)
depomesh["radius"] = np.ones(cyan.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(gray)
submesh["radius"] = np.ones(gray.shape[0])*0.5

# Progress bar is a new feature on master branch
maskglyphed = maskmesh.glyph(scale="radius", geom=geom) # progress_bar=True)
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
p.add_mesh(maskglyphed, color='green')
# p.add_mesh(depoglyphed, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

c:\Users\urcs049\AppData\Local\anaconda3\Lib\site-packages\pyvista\core\utilities\points.py:55: UserWarning: Points is not a float type. This can cause issues when transforming or applying filters. Casting to ``np.float32``. Disable this by passing ``force_float=False``.
  warnings.warn(
c:\Users\urcs049\AppData\Local\anaconda3\Lib\site-packages\pyvista\core\filters\data_set.py:2386: UserWarning: No vector-like data to use for orient. orient will be set to False.
  warnings.warn("No vector-like data to use for orient. orient will be set to False.")


Widget(value='<iframe src="http://localhost:56889/index.html?ui=P_0x2428356b9b0_2&reconnect=auto" class="pyvis…

In [7]:
# cyan = torch.Tensor(np.logical_and(etchfilm[:, :50, :,1]==10, etchfilm[:, :50, :,0]==0)).to_sparse()
cyan = torch.Tensor(etchfilm[:, :50, :,1]!=0).to_sparse()
cyan = cyan.indices().numpy().T

gray = torch.Tensor(etchfilm[:, :50, :,0]!=0).to_sparse()
gray = gray.indices().numpy().T

mask = torch.Tensor(etchfilm[:, :50, :,2]!=0).to_sparse()
mask = mask.indices().numpy().T

maskmesh = pv.PolyData(mask)
maskmesh["radius"] = np.ones(mask.shape[0])*0.5

depomesh = pv.PolyData(cyan)
depomesh["radius"] = np.ones(cyan.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(gray)
submesh["radius"] = np.ones(gray.shape[0])*0.5

# Progress bar is a new feature on master branch
maskglyphed = maskmesh.glyph(scale="radius", geom=geom) # progress_bar=True)
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
p.add_mesh(maskglyphed, color='green')
# p.add_mesh(depoglyphed, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

c:\Users\urcs049\AppData\Local\anaconda3\Lib\site-packages\pyvista\core\utilities\points.py:55: UserWarning: Points is not a float type. This can cause issues when transforming or applying filters. Casting to ``np.float32``. Disable this by passing ``force_float=False``.
  warnings.warn(
c:\Users\urcs049\AppData\Local\anaconda3\Lib\site-packages\pyvista\core\filters\data_set.py:2386: UserWarning: No vector-like data to use for orient. orient will be set to False.
  warnings.warn("No vector-like data to use for orient. orient will be set to False.")


Widget(value='<iframe src="http://localhost:56889/index.html?ui=P_0x24284001cd0_0&reconnect=auto" class="pyvis…

In [8]:
# cyan = torch.Tensor(np.logical_and(etchfilm[:50, :, :,1]==10, etchfilm[:50, :, :,0]==0)).to_sparse()
cyan = torch.Tensor(etchfilm[:50, :, :,1]!=0).to_sparse()
cyan = cyan.indices().numpy().T

gray = torch.Tensor(etchfilm[:50, :, :,0]!=0).to_sparse()
gray = gray.indices().numpy().T

mask = torch.Tensor(etchfilm[:50, :, :,2]!=0).to_sparse()
mask = mask.indices().numpy().T

maskmesh = pv.PolyData(mask)
maskmesh["radius"] = np.ones(mask.shape[0])*0.5

depomesh = pv.PolyData(cyan)
depomesh["radius"] = np.ones(cyan.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(gray)
submesh["radius"] = np.ones(gray.shape[0])*0.5

# Progress bar is a new feature on master branch
maskglyphed = maskmesh.glyph(scale="radius", geom=geom) # progress_bar=True)
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
p.add_mesh(maskglyphed, color='green')
# p.add_mesh(depoglyphed, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

c:\Users\urcs049\AppData\Local\anaconda3\Lib\site-packages\pyvista\core\utilities\points.py:55: UserWarning: Points is not a float type. This can cause issues when transforming or applying filters. Casting to ``np.float32``. Disable this by passing ``force_float=False``.
  warnings.warn(
c:\Users\urcs049\AppData\Local\anaconda3\Lib\site-packages\pyvista\core\filters\data_set.py:2386: UserWarning: No vector-like data to use for orient. orient will be set to False.
  warnings.warn("No vector-like data to use for orient. orient will be set to False.")


Widget(value='<iframe src="http://localhost:56889/index.html?ui=P_0x242834e8aa0_1&reconnect=auto" class="pyvis…

In [ ]:
# cyan = torch.Tensor(np.logical_and(etching[:50, :, :,1]==10, etching[:50, :, :,0]==0)).to_sparse()
cyan = torch.Tensor(etching[:50, :, :,1]!=0).to_sparse()
cyan = cyan.indices().numpy().T

gray = torch.Tensor(etching[:50, :, :,0]!=0).to_sparse()
gray = gray.indices().numpy().T

mask = torch.Tensor(etching[:50, :, :,2]!=0).to_sparse()
mask = mask.indices().numpy().T

maskmesh = pv.PolyData(mask)
maskmesh["radius"] = np.ones(mask.shape[0])*0.5

depomesh = pv.PolyData(cyan)
depomesh["radius"] = np.ones(cyan.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(gray)
submesh["radius"] = np.ones(gray.shape[0])*0.5

# Progress bar is a new feature on master branch
maskglyphed = maskmesh.glyph(scale="radius", geom=geom) # progress_bar=True)
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
p.add_mesh(maskglyphed, color='green')
p.add_mesh(depoglyphed, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()